In [1]:
import sys

!{sys.executable} -W ignore:DEPRECATION -m pip install --quiet duckdb==0.7.1 \
duckdb-engine \
watermark \
ipython-sql \
sqlalchemy \
python-snappy \
pyarrow \
memray \
pandas \
ipywidgets  \
matplotlib \
gensim 

In [50]:
import duckdb
import pandas as pd
from gensim.models import Word2Vec

In [4]:
%load_ext watermark
# Duckdb 0.7.0 offers a bunch of new JSON stuff that I want to test out, checking to see I have the latest
# https://duckdb.org/2023/03/03/json.html
%watermark --iversions

sys: 3.9.12 (main, Mar 26 2022, 15:51:13) 
[Clang 12.0.0 (clang-1200.0.32.29)]



In [5]:
%load_ext sql

In [6]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [8]:
# Create new DB or load existing https://duckdb.org/docs/guides/python/jupyter.html
%sql duckdb:///viberary.duckdb

In [19]:
%sql DESCRIBE select * from read_json_auto('/Users/vicki/viberary/viberary/data/goodreads_books.json',lines='true');

,column_name,column_type,null,key,default,extra
0,isbn,VARCHAR,YES,None,None,None
1,text_reviews_count,VARCHAR,YES,None,None,None
2,series,BIGINT[],YES,None,None,None
3,country_code,VARCHAR,YES,None,None,None
4,language_code,VARCHAR,YES,None,None,None
5,popular_shelves,"STRUCT(count BIGINT, ""name"" VARCHAR)[]",YES,None,None,None
6,asin,VARCHAR,YES,None,None,None
7,is_ebook,VARCHAR,YES,None,None,None
8,average_rating,VARCHAR,YES,None,None,None
9,kindle_asin,VARCHAR,YES,None,None,None


In [27]:
# Create table in DuckDB
%sql CREATE TABLE goodreads as select * from read_json_auto('/Users/vicki/viberary/viberary/data/goodreads_books.json',lines='true');

(duckdb.CatalogException) Catalog Error: Table with name "goodreads" already exists!
[SQL: CREATE TABLE goodreads as select * from read_json_auto('/Users/vicki/viberary/viberary/data/goodreads_books.json',lines='true');]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [9]:
%sql select book_id, title, popular_shelves as ps, description from goodreads limit 10;

,book_id,title,ps,description
0,5333265,W.C. Fields: A Life on Film,"[{'count': 3, 'name': 'to-read'}, {'count': 1,...",
1,1333909,Good Harbor,"[{'count': 2634, 'name': 'to-read'}, {'count':...","Anita Diamant's international bestseller ""The ..."
2,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...","[{'count': 58, 'name': 'to-read'}, {'count': 1...",Omnibus book club edition containing the Ladie...
3,6066819,Best Friends Forever,"[{'count': 7615, 'name': 'to-read'}, {'count':...",Addie Downs and Valerie Adler were eight when ...
4,287140,Runic Astrology: Starcraft and Timekeeping in ...,"[{'count': 32, 'name': 'to-read'}, {'count': 3...",
5,287141,The Aeneid for Boys and Girls,"[{'count': 56, 'name': 'to-read'}, {'count': 1...","Relates in vigorous prose the tale of Aeneas, ..."
6,378460,The Wanting of Levine,"[{'count': 14, 'name': 'to-read'}, {'count': 1...",
7,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,"[{'count': 515, 'name': 'to-read'}, {'count': ...","To Kara's astonishment, she discovers that a p..."
8,34883016,Playmaker: A Venom Series Novella,"[{'count': 4, 'name': 'to-read'}, {'count': 1,...",Secrets. Sometimes keeping them in confidence ...
9,287149,The Devil's Notebook,"[{'count': 961, 'name': 'to-read'}, {'count': ...","Wisdom, humor, and dark observations by the fo..."


In [111]:
# This will not work with SQL magic because of this: 
# https://github.com/catherinedevlin/ipython-sql/blob/master/src/sql/parse.py#L89

query=r"""select concat(' ' , title,description) from goodreads limit 10;"""
sq = shlex.split(query,posix=True)

%sql $sq


(duckdb.ParserException) Parser Error: syntax error at or near "["
LINE 1: ['select', 'concat( ' , ',' , 'title,de...
        ^
[SQL: ['select', 'concat( ' , ',' , 'title,description)' , 'from' , 'goodreads' , 'limit' , '10;' ]]
(Background on this error at: https://sqlalche.me/e/14/f405)


In [123]:
# Let's start with title and description as our sentence feature

# we have to connect with pyscopg because the sql magic doesn't allow string literal concatenation due to Posix=
con = duckdb.connect('viberary.duckdb')
# create a table and load data into it
sentences = con.sql("select concat_ws(' ' , title,description) from goodreads limit 10;").df()

In [136]:
# input for training Word2Vec is a list of lists or iterable
# https://radimrehurek.com/gensim/models/word2vec.html
def gen_rows(df):
    for row in df.itertuples(index=False):
        yield list(row)

In [139]:
model = Word2Vec(sentences, min_count=1,vector_size=100, window=5, workers=4)
model.train
model.save("word2vec.model")

In [142]:
# check to make sure model saved
!ls

00_viberary_json_eda.ipynb memray-results
01_memray_tracing.ipynb    my-db.duckdb
02_parquet_eda.ipynb       my-db.duckdb.db
03_duckdb_eda.ipynb        my-db.duckdb.db.wal
04_word2vec.ipynb          my-db.duckdb.wal
05_duckdb_0.7.1.ipynb      viberary.duckdb
Untitled.ipynb             viberary.duckdb.wal
cbow.ipynb                 word2vec.model
goodreads.csv


In [156]:
#TODO: get all vocabulary
word_vectors = model.wv
word_vectors.load
sims = model.wv.most_similar('a', topn=10)

In [153]:
sims

[('c', 0.12309540063142776),
 ("'", 0.08063945919275284),
 ('d', 0.06539318710565567),
 ('t', 0.05550684407353401),
 ('o', 0.016174763441085815),
 ('(', 0.01423672679811716),
 ('e', 0.011071541346609592),
 ('l', -0.0035478866193443537),
 (',', -0.013670560903847218),
 (' ', -0.019223935902118683)]